In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import time
import copy
import sys
import torch.nn.init as init
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.0.1
Torchvision Version:  0.2.2


In [2]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
print(torch.cuda.get_device_name(0))

GeForce RTX 2080 Ti


In [3]:
runs_list = next(os.walk('../Data/Images'))[1]

In [4]:
path = "../Data/Labels.csv"
header = ['Run','AD']
Labelsdf = pd.read_csv(path, names=header, usecols=[1,2], skiprows=1, index_col=False)

In [5]:
labels_dict = dict(zip(Labelsdf.Run, Labelsdf.AD))

In [6]:
import re

# process then save the 3d image ndarray as binary
# Images of 96, 96, 48 from 256, 176, 256

# Crop in the x and y direction (get center/hippocampi area)
# [80:176, 40:136, ]

img_dir = "../Data/Images"
dest_dir = "../Data/Processed"

if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)
    print("Created ouput directory: " + dest_dir)

    #low_bound = 100
    #high_bound = 200
    #img_scale = 1
    depth_scale = 2

    imgs, labels = [], []

    for run in runs_list:
#         print(run)
        run_dir = os.path.join(img_dir, run)
        run_imgs = []
        for filename in os.listdir(run_dir):
            img_slice = cv2.imread(os.path.join(run_dir, filename), cv2.IMREAD_GRAYSCALE)
            # if image is square? IF DO THIS, NEED TO ACCOUNT FOR CENTER OFF SET (176 X 240)
            if  img_slice is None or img_slice.shape[0] == img_slice.shape[1]:
                print(run)
                print('1')
                break
            img_num = int(filename[-7:-4])
            if (100 <= img_num < 196):
                img_slice = cv2.imread(os.path.join(run_dir, filename), cv2.IMREAD_GRAYSCALE)
                #img_slice = cv2.resize(img_slice, (0,0), fx=1/img_scale, fy=1/img_scale, interpolation=cv2.INTER_AREA)
                img_slice = img_slice[80:176,40:136]
                # normalize pixel intensity to range of 0 and 1
                img_slice = img_slice/256
                run_imgs.append(img_slice)

        if len(run_imgs) == 0 or len(run_imgs) != 96:
            print(run)
            print('2')
            continue

        temp_arr = np.array(run_imgs)

        final_slices = []
        #print(temp_arr.shape[2])

        for y in range(temp_arr.shape[2]):
            xz_pane = temp_arr[:, :, y]
            scaled_xz = cv2.resize(xz_pane, (0, 0), fy=1/depth_scale, fx=1, interpolation=cv2.INTER_AREA)
            final_slices.append(scaled_xz)

        if not len(final_slices[0]) == 48:
            print(run)
            print('3')
            continue

        final_slices = np.dstack(final_slices)
        final_slices = np.expand_dims(final_slices, axis=0)
        #final_array = torch.from_numpy(np.dstack(final_slices)).float()
        if run in labels_dict:
            # save as binary
            np.save(os.path.join(dest_dir, run), final_slices)
            imgs.append(run)
            labels.append(labels_dict[run])
        else:
            continue

    imgs = np.array(imgs)
    labels = np.array(labels)

    np.save(os.path.join(dest_dir, 'imgs'), imgs)
    np.save(os.path.join(dest_dir, 'labels'), labels)

            
else:
    imgs = np.load(os.path.join(dest_dir, 'imgs.npy'))
    labels = np.load(os.path.join(dest_dir, 'labels.npy'))
    

#     subject_regex = re.compile("OAS(?P<order>[0-9]+)")
#     subject = subject_regex.search(run).group(1)
    
#     if int(subject[-1]) < 6:
#         train_data.append(run_tuple)
#     elif 6 <= int(subject[-1]) <=7:
#         valid_data.append(run_tuple)
#     elif 8 <= int(subject[-1]) <= 9:
#         test_data.append(run_tuple)
    
    
    
    
# print("Number of data points in train dataset: {}".format(len(train_data)))
# print("Number of data points in valid dataset: {}".format(len(valid_data)))
# print("Number of data points in test dataset: {}".format(len(test_data)))

    

In [7]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "vgg16bn_custom"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 100

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [8]:
train_imgs, train_labels = [], []
val_imgs, val_labels = [], []
test_imgs, test_labels = [], []

for idx, ad in np.ndenumerate(labels):
    if (idx[0] % 10) < 6:
        train_imgs.append(imgs[idx[0]])
        train_labels.append(ad)
    if ((idx[0] % 10) >= 6) and ((idx[0] % 10) < 8):
        val_imgs.append(imgs[idx[0]])
        val_labels.append(ad)
    if ((idx[0] % 10) >= 8) and ((idx[0] % 10) <= 9):
        test_imgs.append(imgs[idx[0]])
        test_labels.append(ad)
        
train_imgs = np.array(train_imgs)
train_labels = np.array(train_labels)
val_imgs = np.array(val_imgs)
val_labels = np.array(val_labels)
test_imgs = np.array(test_imgs)
test_labels = np.array(test_labels)
        
ad_imgs, ad_labels = [], []

for idx, ad in np.ndenumerate(train_labels):
    if ad:
        ad_imgs.append(imgs[idx[0]])
        ad_labels.append(ad)

ad_imgs = np.array(ad_imgs)
ad_labels = np.array(ad_labels)

normal_ratio = train_imgs.shape[0] // ad_imgs.shape[0]

train_imgs = np.concatenate((train_imgs, np.repeat(ad_imgs, normal_ratio - 1, axis=0)), axis=0)
train_labels = np.concatenate((train_labels, np.repeat(ad_labels, normal_ratio -1, axis=0)), axis=0)

# new_imgs = imgs
# new_labels = labels

In [9]:
class T1Dataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = data
        self.target = torch.from_numpy(target).long()
        self.transform = transform

    def __getitem__(self, index):
#         print(self.data[index])
        x = np.load(os.path.join(dest_dir, self.data[index]) + ".npy")
#         print(x.shape)
        x = torch.from_numpy(x).float()
        y = self.target[index]

        if self.transform:
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

In [10]:
train_dataset = T1Dataset(train_imgs, train_labels)
val_dataset = T1Dataset(val_imgs, val_labels)
test_dataset = T1Dataset(test_imgs, test_labels)

image_datasets = {'train': train_dataset, 'val': val_dataset, 'test': test_dataset}
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                   batch_size=batch_size,
                                                   pin_memory=True,
                                                   shuffle=True, num_workers=0) for x in ['train', 'val', 'test']}

In [11]:
class Paper3DCNN(nn.Module):
    def __init__(self):
        super(Paper3DCNN, self).__init__()
        self.name = "Paper3DCNN"
        # ?x1x96x96x48 to ?x32x96x96x48
        # kernel size is always 3 or 1 in vgg, stride is 1, padding is s.t. the resolution preserved after convolution
        # padding is 1 pixel for 3x3 convolution
        # max pooling 2, 2
        
        # first block
        self.conv1_1 = nn.Conv3d(1, 32, padding=1, kernel_size=3, bias=True)
        # max pool
        
        # second block
        self.conv2_1 = nn.Conv3d(32, 64, padding=1, kernel_size=3, bias=True)
        # max pool
        
        # third block
        self.conv3_1 = nn.Conv3d(64, 128, padding=1, kernel_size=3, bias=True)
        self.conv3_2 = nn.Conv3d(128, 128, padding=1, kernel_size=3, bias=True)
        # max pool
        
        # fourth block
        self.conv4_1 = nn.Conv3d(128, 256, padding=1, kernel_size=3, bias=True)
        self.conv4_2 = nn.Conv3d(256, 256, padding=1, kernel_size=3, bias=True)
        # max pool
        
        # fifth block
        self.conv5_1 = nn.Conv3d(256, 256, padding=1, kernel_size=3, bias=True)
        self.conv5_2 = nn.Conv3d(256, 256, padding=1, kernel_size=3, bias=True)
        # max pool 2
        
        self.maxpool1 = nn.MaxPool3d(2, 2)
        self.maxpool2 = nn.MaxPool3d(kernel_size=(2,2,2), padding=(1,0,0), stride=2)
        
        self.fc1 = nn.Linear(4608, 1024)
        self.fc2 = nn.Linear(1024, 10)
        self.fc3 = nn.Linear(10, 2)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        #residual = x
        
        # first block
        out = self.conv1_1(x)
        out = self.relu(out)
        out = self.maxpool1(out)
        #print(out.shape)
        
        # second block
        out = self.conv2_1(out)
        out = self.relu(out)
        out = self.maxpool1(out)
        #print(out.shape)

        # third block
        out = self.conv3_1(out)
        out = self.relu(out)
        out = self.conv3_2(out)
        out = self.relu(out)
        out = self.maxpool1(out)
        #print(out.shape)

        # fourth block
        out = self.conv4_1(out)
        out = self.relu(out)
        out = self.conv4_2(out)
        out = self.relu(out)
        out = self.maxpool1(out)
        #print(out.shape)
        
        # fourth block
        out = self.conv5_1(out)
        out = self.relu(out)
        out = self.conv5_2(out)
        out = self.relu(out)
        out = self.maxpool2(out)
        #print(out.shape)

        # if self.downsample is not None:
        #     residual = self.downsample(x)

        #out += residual
        out = out.view(-1, 4608)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        #print(out.shape)

        return out

In [12]:
import math

import torch.nn as nn
import torch.nn.init as init

__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]

class VGG(nn.Module):
    '''
    VGG model 
    '''
    def __init__(self, features):
        super(VGG, self).__init__()
#         self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 3 * 3, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 2),
        )
         # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()
        
        self.avgpool = nn.AdaptiveAvgPool3d((1, 3, 3))
        
        self.features3d = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2)), #256
            
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.Conv3d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2)), #128
            
            nn.Conv3d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2)), #64
            
            nn.Conv3d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(inplace=True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2)), #32
            
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(inplace=True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2)), #16
        )

    def forward(self, x):
#         x = torch.reshape(x, (-1, 48, 96, 96))
#         x = self.features(x)
#         x = torch.unsqueeze(x, 1)
        x = self.features3d(x)
#         print(x.shape)
        
#         exit()
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
#         print(x.shape)
        x = self.classifier(x)
        return x


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 48
#     in_channels_3d = 1
#     dimension = '2d' 
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
#         if dimension = '3d':
#             if v == 'M':
#                 layers += [nn.MaxPool3d(kernel_size=(2, 1, 1))]
#             else:
#                 conv3d = nn.Conv3d(in_channels_3d, v, kernel_size=3, padding=1)
#                 if batch_norm:
#                     layers += [conv3d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
#                 else:
#                     layers += [conv2d, nn.ReLU(inplace=True)]
#                 in_channels_3d = v
    return nn.Sequential(*layers)


cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 
          512, 512, 512, 512, 'M'],
}


def vgg11():
    """VGG 11-layer model (configuration "A")"""
    return VGG(make_layers(cfg['A']))


def vgg11_bn():
    """VGG 11-layer model (configuration "A") with batch normalization"""
    return VGG(make_layers(cfg['A'], batch_norm=True))


def vgg13():
    """VGG 13-layer model (configuration "B")"""
    return VGG(make_layers(cfg['B']))


def vgg13_bn():
    """VGG 13-layer model (configuration "B") with batch normalization"""
    return VGG(make_layers(cfg['B'], batch_norm=True))


def vgg16():
    """VGG 16-layer model (configuration "D")"""
    return VGG(make_layers(cfg['D']))


def vgg16_bn():
    """VGG 16-layer model (configuration "D") with batch normalization"""
    return VGG(make_layers(cfg['D'], batch_norm=True))


def vgg19():
    """VGG 19-layer model (configuration "E")"""
    return VGG(make_layers(cfg['E']))


def vgg19_bn():
    """VGG 19-layer model (configuration 'E') with batch normalization"""
    return VGG(make_layers(cfg['E'], batch_norm=True))

In [13]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            normal_corrects, ad_corrects = 0, 0
            normal_count, ad_count = 0, 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
#                 print(inputs.shape)
#                 new1 = torch.reshape(inputs, (-1, 96, 96))
#                 print(new1.shape)
#                 new2 = torch.unsqueeze(new1, 1)
#                 print(new2.shape)
#                 exit()
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
#                         print(outputs, labels)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
#                 print(preds)
#                 print(labels.data)
                correctness = preds == labels.data
                running_corrects += torch.sum(correctness)
                
                
                index = 0
                for condition in labels.data:
                    
                    if condition == 0:
                        if correctness[index] == 1:
                            normal_corrects += 1
                        normal_count += 1
                    if condition == 1:
                        if correctness[index] == 1:
                            ad_corrects += 1
                        ad_count += 1
                    index += 1
                

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            epoch_normal_acc = normal_corrects / normal_count
#             print(normal_corrects, normal_count)
            epoch_ad_acc = ad_corrects / ad_count
#             print(ad_corrects, ad_count)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            print('Normal Acc: {:.4f} AD Acc: {:.4f}'.format(epoch_normal_acc, epoch_ad_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [14]:
# def weight_init(m):
#     '''
#     Usage:
#         model = Model()
#         model.apply(weight_init)
#     '''
#     if isinstance(m, nn.Conv1d):
#         init.normal_(m.weight.data)
#         if m.bias is not None:
#             init.normal_(m.bias.data)
#     elif isinstance(m, nn.Conv2d):
#         init.xavier_normal_(m.weight.data)
#         if m.bias is not None:
#             init.normal_(m.bias.data)
#     elif isinstance(m, nn.Conv3d):
#         init.xavier_normal_(m.weight.data)
#         if m.bias is not None:
#             init.normal_(m.bias.data)
#     elif isinstance(m, nn.ConvTranspose1d):
#         init.normal_(m.weight.data)
#         if m.bias is not None:
#             init.normal_(m.bias.data)
#     elif isinstance(m, nn.ConvTranspose2d):
#         init.xavier_normal_(m.weight.data)
#         if m.bias is not None:
#             init.normal_(m.bias.data)
#     elif isinstance(m, nn.ConvTranspose3d):
#         init.xavier_normal_(m.weight.data)
#         if m.bias is not None:
#             init.normal_(m.bias.data)
#     elif isinstance(m, nn.BatchNorm1d):
#         init.normal_(m.weight.data, mean=1, std=0.02)
#         init.constant_(m.bias.data, 0)
#     elif isinstance(m, nn.BatchNorm2d):
#         init.normal_(m.weight.data, mean=1, std=0.02)
#         init.constant_(m.bias.data, 0)
#     elif isinstance(m, nn.BatchNorm3d):
#         init.normal_(m.weight.data, mean=1, std=0.02)
#         init.constant_(m.bias.data, 0)
#     elif isinstance(m, nn.Linear):
#         init.xavier_normal_(m.weight.data)
#         init.normal_(m.bias.data)
#     elif isinstance(m, nn.LSTM):
#         for param in m.parameters():
#             if len(param.shape) >= 2:
#                 init.orthogonal_(param.data)
#             else:
#                 init.normal_(param.data)
#     elif isinstance(m, nn.LSTMCell):
#         for param in m.parameters():
#             if len(param.shape) >= 2:
#                 init.orthogonal_(param.data)
#             else:
#                 init.normal_(param.data)
#     elif isinstance(m, nn.GRU):
#         for param in m.parameters():
#             if len(param.shape) >= 2:
#                 init.orthogonal_(param.data)
#             else:
#                 init.normal_(param.data)
#     elif isinstance(m, nn.GRUCell):
#         for param in m.parameters():
#             if len(param.shape) >= 2:
#                 init.orthogonal_(param.data)
#             else:
#                 init.normal_(param.data)

In [15]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [16]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 256

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 256

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 256

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 256

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 256

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    elif model_name == "Paper3DCNN":
        model_ft = Paper3DCNN()
        
    elif model_name == "vgg16bn_custom":
        model_ft = vgg16_bn()
        
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# # Initialize the model for this run
# model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# # Print the model we just instantiated
# print(model_ft)

In [17]:
scratch_model,_ = initialize_model(model_name, num_classes, feature_extract=False, use_pretrained=False)
print(scratch_model)
# scratch_model.apply(weight_init)
scratch_model = scratch_model.to(device)
scratch_optimizer = optim.SGD(scratch_model.parameters(), lr=0.01, momentum=0.9)
scratch_criterion = nn.CrossEntropyLoss()
_,scratch_hist = train_model(scratch_model, dataloaders_dict, scratch_criterion, scratch_optimizer, num_epochs=num_epochs, is_inception=(model_name=="inception"))

VGG(
  (classifier): Sequential(
    (0): Linear(in_features=4608, out_features=4096, bias=True)
    (1): ReLU(inplace)
    (2): Dropout(p=0.5)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU(inplace)
    (5): Dropout(p=0.5)
    (6): Linear(in_features=4096, out_features=2, bias=True)
  )
  (avgpool): AdaptiveAvgPool3d(output_size=(1, 3, 3))
  (features3d): Sequential(
    (0): Conv3d(1, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (7): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (8): BatchNorm3d(1

Normal Acc: 0.4329 AD Acc: 0.8804

Epoch 35/99
----------
train Loss: 0.4992 Acc: 0.7646
Normal Acc: 0.7139 AD Acc: 0.8112
val Loss: 0.6287 Acc: 0.6318
Normal Acc: 0.6645 AD Acc: 0.4674

Epoch 36/99
----------
train Loss: 0.4689 Acc: 0.7879
Normal Acc: 0.7153 AD Acc: 0.8546
val Loss: 0.8237 Acc: 0.4170
Normal Acc: 0.3550 AD Acc: 0.7283

Epoch 37/99
----------
train Loss: 0.4578 Acc: 0.8002
Normal Acc: 0.7465 AD Acc: 0.8495
val Loss: 0.5463 Acc: 0.7419
Normal Acc: 0.7814 AD Acc: 0.5435

Epoch 38/99
----------
train Loss: 0.4204 Acc: 0.8185
Normal Acc: 0.7500 AD Acc: 0.8814
val Loss: 0.7646 Acc: 0.5866
Normal Acc: 0.5519 AD Acc: 0.7609

Epoch 39/99
----------
train Loss: 0.3933 Acc: 0.8348
Normal Acc: 0.7722 AD Acc: 0.8922
val Loss: 0.6681 Acc: 0.7076
Normal Acc: 0.7641 AD Acc: 0.4239

Epoch 40/99
----------
train Loss: 0.3875 Acc: 0.8434
Normal Acc: 0.7799 AD Acc: 0.9018
val Loss: 0.4698 Acc: 0.7581
Normal Acc: 0.7900 AD Acc: 0.5978

Epoch 41/99
----------
train Loss: 0.3736 Acc: 0.8467

KeyboardInterrupt: 

In [18]:
# Plot the training curves of validation accuracy vs. number
#  of training epochs for the transfer learning method and
#  the model trained from scratch
shist = []

shist = [h.cpu().numpy() for h in scratch_hist]

plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
plt.plot(range(1,num_epochs+1),shist,label="Scratch")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.show()

NameError: name 'scratch_hist' is not defined

In [ ]:
# t = torch.tensor([[[[[0, 1], [2, 3]], [[4, 5], [6, 7]], [[8, 9], [10, 11]]]], 
#                  [[[[12, 13], [14, 15]], [[16, 17], [18, 19]], [[20, 21], [22, 23]]]]])
# print(t)
# print(t.shape)

In [ ]:
# print(torch.reshape(t, (-1, 2, 2)))

In [ ]:
# print(torch.reshape(t, (-1, 2, 2)).shape)

In [ ]:
# [32, 512, 3, 3]